In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
import numpy as np
import os

In [2]:
os.makedirs("Output", exist_ok=True)

In [3]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()


In [4]:
x_train = x_train / 255.0
x_test = x_test / 255.0

In [5]:
class_names = [
    'airplane', 'automobile', 'bird', 'cat', 'deer',
    'dog', 'frog', 'horse', 'ship', 'truck'
]

In [7]:
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1)
])

In [8]:
model = models.Sequential([
    data_augmentation,
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(32,32,3)),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(64, (3,3), activation='relu'),

    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')
])

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [9]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [11]:
history = model.fit(
    x_train, y_train,
    epochs=3,
    validation_data=(x_test, y_test)
)


Epoch 1/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 111s 71ms/step - accuracy: 0.3642 - loss: 1.7377 - val_accuracy: 0.5157 - val_loss: 1.3440
Epoch 2/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 90s 57ms/step - accuracy: 0.5035 - loss: 1.3824 - val_accuracy: 0.5821 - val_loss: 1.1738
Epoch 3/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 98s 63ms/step - accuracy: 0.5562 - loss: 1.2494 - val_accuracy: 0.5990 - val_loss: 1.1163


In [12]:
test_loss, test_accuracy = model.evaluate(x_test, y_test)


313/313 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - accuracy: 0.6050 - loss: 1.1041


In [13]:
with open("Output/model_accuracy.txt", "w") as f:
    f.write(f"Test Accuracy: {test_accuracy}\n")
    f.write(f"Test Loss: {test_loss}\n")


In [14]:
with open("Output/training_log.txt", "w") as f:
    for i in range(len(history.history['accuracy'])):
        f.write(
            f"Epoch {i+1} | "
            f"Train Accuracy: {history.history['accuracy'][i]} | "
            f"Validation Accuracy: {history.history['val_accuracy'][i]}\n"
        )

In [15]:
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.savefig("Output/accuracy_plot.png")
plt.close()

In [16]:
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.savefig("Output/loss_plot.png")
plt.close()


In [17]:
predictions = model.predict(x_test[:9])

plt.figure(figsize=(6,6))
for i in range(9):
    plt.subplot(3,3,i+1)
    plt.imshow(x_test[i])
    plt.title(class_names[np.argmax(predictions[i])])
    plt.axis('off')

plt.tight_layout()
plt.savefig("Output/sample_predictions.png")
plt.close()

print("Training completed. Outputs saved in Output folder.")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 236ms/step
Training completed. Outputs saved in Output folder.
